In [2]:
import pandas as pd
import numpy as np

# Load results parquet
df = pd.read_parquet("gitarchive_results_jan2015_nov2025.parquet")

# Load org IDs
org_ids = pd.read_csv("org_id_only.csv")["org_id"]

df.head()


,org_id,month,push_requests_per_month,commits_per_month,active_developers_per_month,forks_per_month,forks_per_repo_per_month,total_stars_per_month,active_repos_per_month,changed_to_public_repo_per_month,public_repos_per_month,pull_requests_created_per_month,pull_requests_merged_per_month
0,128,2015-01,17,28,244,37,3.083333,182,26,0,0,20,9
1,128,2015-02,21,39,206,20,2.000000,159,21,0,0,8,4
2,128,2015-03,24,120,228,28,4.000000,180,23,0,0,10,8
3,128,2015-04,78,170,226,34,3.090909,164,20,0,2,19,15
4,128,2015-05,12,16,177,16,2.666667,127,13,0,0,6,2


In [2]:
print("=== ORG_ID SUMMARY ===")

total = len(org_ids)
unique = org_ids.nunique()
duplicates = total - unique

print(f"Total org_ids in source file: {total}")
print(f"Unique org_ids: {unique}")
print(f"Duplicate org_ids: {duplicates}")
print("All values in source org ids are numeric:", pd.api.types.is_numeric_dtype(org_ids))

=== ORG_ID SUMMARY ===
Total org_ids in source file: 113231
Unique org_ids: 113231
Duplicate org_ids: 0
All values in source org ids are numeric: True


In [3]:
print("=== ORG ID SANITY CHECK ===")

df_orgs = set(df["org_id"].unique())
input_orgs = set(org_ids)

missing_in_df = input_orgs - df_orgs
unexpected_in_df = df_orgs - input_orgs

print(f"Number of org_ids expected: {len(input_orgs)}")
print(f"Number found in query output: {len(df_orgs)}")
print(f"Missing orgs id in query output: {missing_in_df if missing_in_df else 'None'}")
print(f"Unexpected orgs in query output: {unexpected_in_df if unexpected_in_df else 'None'}")


=== ORG ID SANITY CHECK ===
Number of org_ids expected: 113231
Number found in query output: 113231
Missing orgs id in query output: None
Unexpected orgs in query output: None


In [4]:
print("=== QUERY DATA INFO ===")
df.info()


=== QUERY DATA INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14833261 entries, 0 to 14833260
Data columns (total 13 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   org_id                            Int64  
 1   month                             object 
 2   push_requests_per_month           Int64  
 3   commits_per_month                 Int64  
 4   active_developers_per_month       Int64  
 5   forks_per_month                   Int64  
 6   forks_per_repo_per_month          float64
 7   total_stars_per_month             Int64  
 8   active_repos_per_month            Int64  
 9   changed_to_public_repo_per_month  Int64  
 10  public_repos_per_month            Int64  
 11  pull_requests_created_per_month   Int64  
 12  pull_requests_merged_per_month    Int64  
dtypes: Int64(11), float64(1), object(1)
memory usage: 1.6+ GB


In [5]:
print("=== DATE RANGE CHECK ===")

month_min = df["month"].min()
month_max = df["month"].max()

print(f"DF Range: {month_min} → {month_max}")


=== DATE RANGE CHECK ===
DF Range: 2015-01 → 2025-11


In [6]:
print("=== MISSING VALUES ===")

missing = df.isnull().sum()
display(missing)

=== MISSING VALUES ===


org_id                              0
month                               0
push_requests_per_month             0
commits_per_month                   0
active_developers_per_month         0
forks_per_month                     0
forks_per_repo_per_month            0
total_stars_per_month               0
active_repos_per_month              0
changed_to_public_repo_per_month    0
public_repos_per_month              0
pull_requests_created_per_month     0
pull_requests_merged_per_month      0
dtype: int64

In [7]:
metric_cols = [
    "push_requests_per_month",
    "commits_per_month",
    "active_developers_per_month",
    "forks_per_month",
    "forks_per_repo_per_month",
    "total_stars_per_month",
    "active_repos_per_month",
    "changed_to_public_repo_per_month",
    "public_repos_per_month",
    "pull_requests_created_per_month",
    "pull_requests_merged_per_month"
]

print("=== MEMORY SAFE SUMMARY WITH QUANTILES ===")

percentiles = [0.25, 0.5, 0.75]
summary_rows = []

for col in metric_cols:
    s = df[col]

    # Base stats
    col_summary = {
        "column": col,
        "mean": s.mean(),
        "median": s.median(),
        "min": s.min(),
        "max": s.max(),
    }

    # Quantiles (pandas quantile is optimized & safe for big data)
    q = s.quantile(percentiles)

    for p in percentiles:
        col_summary[f"q{int(p*100)}"] = q[p]

    summary_rows.append(col_summary)

summary_df = pd.DataFrame(summary_rows)
summary_df




=== MEMORY SAFE SUMMARY WITH QUANTILES ===


,column,mean,median,min,max,q25,q50,q75
0,push_requests_per_month,15.087880,0.0,0.0,2388420.0,0.0,0.0,0.0
1,commits_per_month,27.439399,0.0,0.0,5682020.0,0.0,0.0,0.0
2,active_developers_per_month,3.465752,0.0,0.0,25012.0,0.0,0.0,0.0
3,forks_per_month,0.697078,0.0,0.0,11177.0,0.0,0.0,0.0
4,forks_per_repo_per_month,0.186626,0.0,0.0,1440.0,0.0,0.0,0.0
5,total_stars_per_month,2.130851,0.0,0.0,57438.0,0.0,0.0,0.0
6,active_repos_per_month,1.006660,0.0,0.0,22757.0,0.0,0.0,0.0
7,changed_to_public_repo_per_month,0.022998,0.0,0.0,15596.0,0.0,0.0,0.0
8,public_repos_per_month,0.074691,0.0,0.0,3508.0,0.0,0.0,0.0
9,pull_requests_created_per_month,1.885700,0.0,0.0,51624.0,0.0,0.0,0.0


In [8]:
print("=== TIME SERIES COMPLETENESS ===")

expected_months = len(pd.date_range("2015-01-01", "2025-11-01", freq="MS"))

month_counts = (
    df.groupby("org_id")["month"]
      .nunique()
      .reset_index(name="month_count")
)

display(month_counts)

incomplete = month_counts[month_counts["month_count"] < expected_months]

if incomplete.empty:
    print("\n✔ All orgs have complete month coverage.")
else:
    print("\n Orgs with missing months:")
    display(incomplete)


=== TIME SERIES COMPLETENESS ===


,org_id,month_count
0,128,131
1,359,131
2,1067,131
3,1190,131
4,1511,131
...,...,...
113226,121659210,131
113227,121669209,131
113228,121675404,131
113229,121695906,131



✔ All orgs have complete month coverage.
